In [2]:
import os
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
from pyspark.sql.functions import col
from pyspark.sql.functions import expr
from pyspark.sql.functions import when, isnull

In [3]:
#Create SparkSession",
if 'spark' not in globals():
    spark = SparkSession.builder.appName('nycparking').getOrCreate()
else:
    print(f'Spark {spark.version} is already initalized.')

23/02/11 02:30:42 WARN Utils: Your hostname, DESKTOP-K4JEK0C resolves to a loopback address: 127.0.1.1; using 172.29.234.4 instead (on interface eth0)
23/02/11 02:30:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/11 02:30:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/11 02:30:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/02/11 02:30:45 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/02/11 02:30:45 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/02/11 02:30:45 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [5]:
DATA_DIR = "/mnt/c/Users/raveendra sawkar/Documents/IITH Docs/IIITH Assgn/Spark/"
original_df = spark.read.json(os.path.join(DATA_DIR, 'iot_devices.json'))
original_df.limit(5).toPandas().head()

,battery_level,c02_level,cca2,cca3,cn,device_id,device_name,humidity,ip,latitude,lcd,longitude,scale,temp,timestamp
0,8,868,US,USA,United States,1,meter-gauge-1xbYRYcj,51,68.161.225.1,38.00,green,-97.00,Celsius,34,1458444054093
1,7,1473,NO,NOR,Norway,2,sensor-pad-2n2Pea,70,213.161.254.1,62.47,red,6.15,Celsius,11,1458444054119
2,2,1556,IT,ITA,Italy,3,device-mac-36TWSKiT,44,88.36.5.1,42.83,red,12.83,Celsius,19,1458444054120
3,6,1080,US,USA,United States,4,sensor-pad-4mzWkz,32,66.39.173.154,44.06,yellow,-121.32,Celsius,28,1458444054121
4,4,931,PH,PHL,Philippines,5,therm-stick-5gimpUrBB,62,203.82.41.9,14.58,green,120.97,Celsius,25,1458444054122


In [6]:
original_df.createOrReplaceTempView("iot")

In [7]:
spark.sql("select device_id, device_name from iot").show()

+---------+--------------------+
|device_id|         device_name|
+---------+--------------------+
|        1|meter-gauge-1xbYRYcj|
|        2|   sensor-pad-2n2Pea|
|        3| device-mac-36TWSKiT|
|        4|   sensor-pad-4mzWkz|
|        5|therm-stick-5gimp...|
|        6|sensor-pad-6al7RT...|
|        7|meter-gauge-7GeDoanM|
|        8|sensor-pad-8xUD6p...|
|        9| device-mac-9GcjZ2pw|
|       10|sensor-pad-10Bsyw...|
|       11|meter-gauge-11dlM...|
|       12|sensor-pad-12Y2kIm0o|
|       13|meter-gauge-13Gro...|
|       14|sensor-pad-14QL93...|
|       15|  device-mac-15se6mZ|
|       16|sensor-pad-16aXmI...|
|       17|meter-gauge-17zb8...|
|       18|sensor-pad-18XULN9Xv|
|       19|meter-gauge-19eg1...|
|       20|sensor-pad-20gFNf...|
+---------+--------------------+
only showing top 20 rows



In [12]:
spark.sql("SELECT COUNT(device_id) AS Countrywise_Device_Count, cca3 FROM iot GROUP BY cca3 ORDER BY Countrywise_Device_Count DESC").show()

+------------------------+----+
|Countrywise_Device_Count|cca3|
+------------------------+----+
|                   70405| USA|
|                   14455| CHN|
|                   12100| JPN|
|                   11879| KOR|
|                    7942| DEU|
|                    6486| GBR|
|                    6041| CAN|
|                    5989| RUS|
|                    5305| FRA|
|                    3224| BRA|
|                    3119| AUS|
|                    2915| ITA|
|                    2880| SWE|
|                    2744| POL|
|                    2488| NLD|
|                    2310| ESP|
|                    2128| TWN|
|                    1867| IND|
|                    1507| CZE|
|                    1487| NOR|
+------------------------+----+
only showing top 20 rows



In [47]:
spark.sql("SELECT cca3, MAX(c02_level) as MaxCO2 FROM iot GROUP BY cca3 ORDER BY MaxCO2 DESC").show(50)

+----+------+
|cca3|MaxCO2|
+----+------+
| LVA|  1599|
| MEX|  1599|
| BOL|  1599|
| ITA|  1599|
| UKR|  1599|
| GBR|  1599|
| ZAF|  1599|
| AUS|  1599|
| KNA|  1599|
| HUN|  1599|
| THA|  1599|
| NOR|  1599|
| FIN|  1599|
| PER|  1599|
| NLD|  1599|
| TUR|  1599|
| USA|  1599|
| BRA|  1599|
| FRA|  1599|
| MYS|  1599|
| CAN|  1599|
| RUS|  1599|
| ROU|  1599|
| ARG|  1599|
| DNK|  1599|
| ESP|  1599|
| IRL|  1599|
| SWE|  1599|
| IND|  1599|
| KAZ|  1599|
| HKG|  1599|
| CHN|  1599|
| DEU|  1599|
| BGR|  1599|
| JPN|  1599|
| CHE|  1599|
| PHL|  1599|
| CZE|  1599|
| EGY|  1599|
| VNM|  1599|
| ARE|  1599|
| KOR|  1599|
| BMU|  1599|
| TWN|  1599|
| POL|  1599|
| GTM|  1598|
| QAT|  1598|
| CRI|  1598|
| MLT|  1598|
| HTI|  1598|
+----+------+
only showing top 50 rows



In [66]:
spark.sql("SELECT device_id, cca3, c02_level, lcd FROM iot WHERE lcd=='red' GROUP BY cca3, c02_level, device_id, lcd ORDER BY c02_level DESC, cca3, device_id ASC").show()

+---------+----+---------+---+
|device_id|cca3|c02_level|lcd|
+---------+----+---------+---+
|    82970| ARE|     1599|red|
|   124391| ARG|     1599|red|
|     4014| AUS|     1599|red|
|    41519| AUS|     1599|red|
|    55417| AUS|     1599|red|
|    65035| AUS|     1599|red|
|    88905| AUS|     1599|red|
|    96713| BGR|     1599|red|
|   104734| BGR|     1599|red|
|   114119| BGR|     1599|red|
|   122125| BGR|     1599|red|
|   195717| BGR|     1599|red|
|    91456| BMU|     1599|red|
|   184715| BOL|     1599|red|
|    16213| BRA|     1599|red|
|    43630| BRA|     1599|red|
|   140606| BRA|     1599|red|
|   144896| BRA|     1599|red|
|   168097| BRA|     1599|red|
|   193442| BRA|     1599|red|
+---------+----+---------+---+
only showing top 20 rows



In [68]:
spark.sql("SELECT device_id, cca3, battery_level FROM iot WHERE battery_level==0 GROUP BY cca3, device_id, battery_level ORDER BY cca3, device_id").show()

+---------+----+-------------+
|device_id|cca3|battery_level|
+---------+----+-------------+
|   122629| AFG|            0|
|   162516| AFG|            0|
|   167828| AFG|            0|
|    65166| AGO|            0|
|   122109| AIA|            0|
|   146980| AIA|            0|
|    96404| ALA|            0|
|   124665| ALA|            0|
|   197516| ALA|            0|
|    12532| ALB|            0|
|    82389| ALB|            0|
|   138223| ALB|            0|
|     6353| ARE|            0|
|    32595| ARE|            0|
|    62126| ARE|            0|
|    64153| ARE|            0|
|    66197| ARE|            0|
|    74301| ARE|            0|
|    77008| ARE|            0|
|    81355| ARE|            0|
+---------+----+-------------+
only showing top 20 rows

